In [6]:
import re
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
service = Service('/Your_path/chromedriver_linux64/chromedriver')
driver = webdriver.Chrome(service=Service())
wait = WebDriverWait(driver, 10)

In [8]:
pattern = r'\((.*?)\)'
search_url = 'https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250'

In [9]:
ul_class_name = 'ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 eBRbsI compact-list-view ipc-metadata-list--base'
li_class_name = 'ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent'
title_class_name = 'ipc-title__text'
type_class_name = 'sc-b189961a-3 hidKPx cli-title-type-data'
review_class_name = 'ipc-rating-star--voteCount'
rating_class_name = 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'


In [10]:
driver.get(search_url)
time.sleep(30)

In [13]:
tv_shows = []

while True:
    # Get the HTML content of the page
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    # Find the main element containing the list of TV shows
    shows_element = soup.find('ul', attrs={'class': ul_class_name})
    li_elements = shows_element.findAll('li', attrs={'class': li_class_name})
    
    print(f"Number of shows found: {len(li_elements)}")
    
    # Loop through each list item to extract data
    for li in li_elements:
        
        # Extract the title of the show
        title_element = li.find('h3', attrs={'class': title_class_name})
        title = title_element.get_text() if title_element else 'N/A'

        # Extract the type of the show (e.g., TV series, miniseries)
        type_element = li.find('span', attrs={'class': type_class_name })
        show_type = type_element.get_text() if type_element else 'N/A'

        # Extract the review count and use regex to clean it
        review_count_element = li.find('span', attrs={'class': review_class_name})
        review = review_count_element.get_text()
        match = re.search(pattern, review)
        review_count = match.group(1) if match else 'N/A'

        # Extract the rating of the show
        rating_element = li.find('span', attrs={'class': rating_class_name})
        rating = rating_element['aria-label'] if type_element else 'N/A'

        # Store the extracted data in a dictionary
        show_data = {
            'title': title,
            'show_type': show_type,
            'rating': rating,
            'review_count': review_count,
        }

        # Print and append the show data
        print(show_data)
        tv_shows.append(show_data)
    
    driver.close()
    break

print("Scraping completed.")


Number of shows found: 250
{'title': '1. Breaking Bad', 'show_type': 'TV Series', 'rating': 'IMDb rating: 9.5', 'review_count': '2.2M'}
{'title': '2. Planet Earth II', 'show_type': 'TV Mini Series', 'rating': 'IMDb rating: 9.5', 'review_count': '160K'}
{'title': '3. Planet Earth', 'show_type': 'TV Mini Series', 'rating': 'IMDb rating: 9.4', 'review_count': '222K'}
{'title': '4. Band of Brothers', 'show_type': 'TV Mini Series', 'rating': 'IMDb rating: 9.4', 'review_count': '535K'}
{'title': '5. Chernobyl', 'show_type': 'TV Mini Series', 'rating': 'IMDb rating: 9.3', 'review_count': '881K'}
{'title': '6. The Wire', 'show_type': 'TV Series', 'rating': 'IMDb rating: 9.3', 'review_count': '382K'}
{'title': '7. Avatar: The Last Airbender', 'show_type': 'TV Series', 'rating': 'IMDb rating: 9.3', 'review_count': '380K'}
{'title': '8. Blue Planet II', 'show_type': 'TV Mini Series', 'rating': 'IMDb rating: 9.3', 'review_count': '47K'}
{'title': '9. The Sopranos', 'show_type': 'TV Series', 'ratin

In [12]:
pd.DataFrame(tv_shows).to_csv('movies.csv', index=False)

Try below code.

In [ ]:
def xpath_soup(element):
    components = []
    target = element if element.name else element.parent
    for node in (target, *target.parents)[-2::-1]:
        tag = '%s:%s' % (node.prefix, node.name) if node.prefix else node.name
        siblings = node.parent.find_all(tag, recursive=False)
        components.append(tag if len(siblings) == 1 else '%s[%d]' % (tag, next(
            index
            for index, sibling in enumerate(siblings, 1)
            if sibling is node
            )))
    return '/%s' % '/'.join(components)

In [ ]:
tv_shows = []


while True:
 
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    shows_element = soup.find('ul', attrs={'class': ul_class_name})

    li_elements = shows_element.findAll('li', attrs={'class': li_class_name})
    
    print(f"Number of shows found: {len(li_elements)}")
    
    j = 1
    for li in li_elements:
        try:
            driver.execute_script("window.scrollBy(0, 200);")

            title_element = li.find('h3', attrs={'class': title_class_name})
            title = title_element.get_text() if title_element else 'N/A'

            type_element = li.find('span', attrs={'class': type_class_name})
            show_type = type_element.get_text() if type_element else 'N/A'

            review_count_element = li.find('span', attrs={'class': review_class_name})
            review = review_count_element.get_text()
            
       
            match = re.search(pattern, review)
            review_count = match.group(1) if match else 'N/A'

            # Clicking on the show button
            show_button_xpath = f'/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{j}]/div[3]/button'
            driver.find_element(By.XPATH, show_button_xpath).click()
            time.sleep(10)

            # Parsing popup content
            content = driver.page_source
            soup_popup = BeautifulSoup(content, 'html.parser')
            show_popup = soup_popup.find('div', attrs={'class': 'ipc-promptable-base__focus-lock'})
            
            rating_element = show_popup.find('span', attrs={'class': 'ipc-rating-star ipc-rating-star--baseAlt ipc-rating-star--imdb btp_rt_ds'})
            rating = rating_element.get_text() if rating_element else 'N/A'

            creator_element = show_popup.find('a', attrs={'class': 'ipc-link ipc-link--baseAlt sc-9bca7e5d-0 ktChvQ'})
            creator = creator_element.get_text() if creator_element else 'N/A'

            show_data = {
                'title': title,
                'show_type': show_type,
                'review_count': review_count,
                'creator': creator,
                'rating': rating
            }

            print(show_data)
            tv_shows.append(show_data)

            # Closing the popup
            close_btn = soup_popup.find('button', attrs={'class': 'ipc-icon-button ipc-icon-button--baseAlt ipc-icon-button--onBase'})
            xpath_close_btn = xpath_soup(close_btn)
            driver.find_element(By.XPATH, xpath_close_btn).click()
            time.sleep(5)

            j += 1
        except Exception as e:
            continue
    break

print("Scraping completed.")


In [ ]:
pd.DataFrame(tv_shows).to_csv('movies.csv', index=False)